In [ ]:
%load_ext autoreload
%autoreload 2
%cd ..

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from src.losses import SoftDiceLoss
from src.models import UNet
from src.data import PlethoraDataModule

In [17]:
input = torch.rand(16, 2, 4, 4)

target = torch.randn(16, 2, 4, 4)
target = torch.argmax(target, dim=1)

In [18]:
SoftDiceLoss(2)(input, target)

torch.Size([16, 2, 4, 4]) torch.Size([16, 4, 4])


tensor(0.5187)

In [25]:
data_module_args = { 
   "batch_size": 8,
   "img_size": 512,
   "clip_low": -1000,
   "clip_high": 1000,
   "pin_memory": True,
   "num_workers": 2
}                                                              
dm = PlethoraDataModule(**data_module_args)
dm.setup()

In [26]:
sample = dm.train_ds[60]
img, mask = sample['img'], sample['mask']

In [3]:
device = "cuda:7"
from src.models.unet import UNet
net = UNet.load_from_checkpoint("logs/unet-plethora-512/version_2/ckpts/last.ckpt")
net.to(device).eval();
net.loss_fn

In [37]:
with torch.no_grad():
    logits = net(img.unsqueeze(0).to(device))

In [50]:
SoftDiceLoss(2)(logits, mask.unsqueeze(0).to(device))

torch.Size([1, 2, 512, 512]) torch.Size([1, 512, 512])


tensor(0.0080, device='cuda:0')

In [9]:
from src.data import PlethoraDataset
ds = PlethoraDataset(ct_dir="data/raw/NSCLC-Radiomics/",
                     mask_dir="data/processed/plethora_masks/")
                     

Caching CT scans metadata: 100%|██████████| 399/399 [04:11<00:00,  1.59it/s]
